# Initialization

In [1]:
import logging

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

# Загрузка данных

In [3]:
items = pd.read_parquet("items.par")
events = pd.read_parquet("events.par")

# Разбиение с учётом хронологии

Рекомендательные системы на практике работают с учётом хронологии. Поэтому поток событий для тренировки и валидации полезно делить на то, что уже случилось, и что ещё случится. Это позволяет проводить валидацию на тех же пользователях, на которых тренировались, но на их событиях в будущем.

# === Знакомство: "холодный" старт

In [4]:
# зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime("2017-08-01").date()

train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

# количество пользователей в train и test
users_train = events_train["user_id"].drop_duplicates()
users_test = events_test["user_id"].drop_duplicates()
# количество пользователей, которые есть и в train, и в test
common_users = set(users_train) & set(users_test)

print(len(users_train), len(users_test), len(common_users)) 

428220 123223 120858


In [5]:
# Определяем холодных пользователей (те, кто есть в test, но нет в train)
cold_users = set(users_test) - set(users_train)

# Выводим количество холодных пользователей
print(len(cold_users))

2365


In [6]:
from sklearn.preprocessing import MinMaxScaler

# Определяем начальную дату для анализа популярности
top_pop_start_date = pd.to_datetime("2015-01-01").date()

# Вычисляем популярность книг на основе количества уникальных пользователей и среднего рейтинга
item_popularity = events_train \
    .query("started_at >= @top_pop_start_date") \
    .groupby("item_id") \
    .agg(users=("user_id", "nunique"), avg_rating=("rating", "mean")) \
    .reset_index()

# Нормализуем количество пользователей и средний рейтинг
scaler = MinMaxScaler()
item_popularity[["users_norm", "avg_rating_norm"]] = scaler.fit_transform(
    item_popularity[["users", "avg_rating"]]
)

# Вычисляем score популярности (учет количества пользователей и среднего рейтинга)
item_popularity["popularity_score"] = (
    item_popularity["users_norm"] * item_popularity["avg_rating_norm"]
)

# Сортируем книги по убыванию popularity_score
item_popularity = item_popularity.sort_values(by="popularity_score", ascending=False)

# Отбираем топ-100 книг со средней оценкой >= 4
top_k_pop_items = item_popularity[item_popularity["avg_rating"] >= 4].head(100)

# Выводим результат
top_k_pop_items.head()

,item_id,users,avg_rating,users_norm,avg_rating_norm,popularity_score
32387,18007564,20207,4.321275,0.496596,0.830319,0.412333
32623,18143977,19462,4.290669,0.478287,0.822667,0.393471
2,3,15139,4.706057,0.372042,0.926514,0.344702
30695,16096824,16770,4.301014,0.412126,0.825253,0.340108
1916,15881,13043,4.632447,0.320529,0.908112,0.291076


In [7]:
# добавляем информацию о книгах
top_k_pop_items = top_k_pop_items.merge(
    items.set_index("item_id")[["author", "title", "genre_and_votes", "publication_year"]], on="item_id")

with pd.option_context('display.max_rows', 100):
    display(top_k_pop_items[["item_id", "author", "title", "publication_year", "users", "avg_rating", "popularity_score", "genre_and_votes"]])

,item_id,author,title,publication_year,users,avg_rating,popularity_score,genre_and_votes
0,18007564,Andy Weir,The Martian,2014,20207,4.321275,0.412333,"{'Science Fiction': 11966, 'Fiction': 8430}"
1,18143977,Anthony Doerr,All the Light We Cannot See,2014,19462,4.290669,0.393471,"{'Historical-Historical Fiction': 13679, 'Fict..."
2,3,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,1997,15139,4.706057,0.344702,"{'Fantasy': 59818, 'Fiction': 17918, 'Young Ad..."
3,16096824,Sarah J. Maas,A Court of Thorns and Roses (A Court of Thorns...,2015,16770,4.301014,0.340108,"{'Fantasy': 14326, 'Young Adult': 4662, 'Roman..."
4,15881,"J.K. Rowling, Mary GrandPré",Harry Potter and the Chamber of Secrets (Harry...,1999,13043,4.632447,0.291076,"{'Fantasy': 50130, 'Young Adult': 15202, 'Fict..."
5,38447,Margaret Atwood,The Handmaid's Tale,1998,14611,4.232770,0.290194,"{'Fiction': 15424, 'Classics': 9937, 'Science ..."
6,11235712,Marissa Meyer,"Cinder (The Lunar Chronicles, #1)",2012,14348,4.179189,0.280247,"{'Young Adult': 10539, 'Fantasy': 9237, 'Scien..."
7,17927395,Sarah J. Maas,A Court of Mist and Fury (A Court of Thorns an...,2016,12177,4.730640,0.279094,"{'Fantasy': 10186, 'Romance': 3346, 'Young Adu..."
8,5,"J.K. Rowling, Mary GrandPré",Harry Potter and the Prisoner of Azkaban (Harr...,2004,11890,4.770143,0.275401,"{'Fantasy': 49784, 'Young Adult': 15393, 'Fict..."
9,13206900,Marissa Meyer,"Winter (The Lunar Chronicles, #4)",2015,12291,4.534293,0.266881,"{'Fantasy': 4835, 'Young Adult': 4672, 'Scienc..."


# === Знакомство: первые персональные рекомендации

In [8]:
# Объединяем данные по item_id, добавляя avg_rating из топ-100 популярных книг
cold_users_events_with_recs = (
    events_test[events_test["user_id"].isin(cold_users)]
    .merge(top_k_pop_items[["item_id", "avg_rating"]], on="item_id", how="left")
)

# Определяем индексы событий, для которых нет avg_rating
cold_user_items_no_avg_rating_idx = cold_users_events_with_recs["avg_rating"].isnull()

# Оставляем только те записи, у которых есть avg_rating
cold_user_recs = cold_users_events_with_recs[~cold_user_items_no_avg_rating_idx] \
    [["user_id", "item_id", "rating", "avg_rating"]]

# Выводим первые строки для проверки
cold_user_recs.head()

,user_id,item_id,rating,avg_rating
14,1000504,1885,5,4.316316
16,1000712,13496,5,4.440779
21,1001508,1885,5,4.316316
24,1002222,15839976,4,4.150180
26,1002222,18966819,5,4.374914


In [9]:
# Общее количество событий холодных пользователей в test
total_cold_events = len(cold_users_events_with_recs)

# Количество событий, для которых найдена книга из top_k_pop_items (т.е. не null в avg_rating)
matched_cold_events = len(cold_user_recs)

# Доля совпадений
match_ratio = matched_cold_events / total_cold_events

# Округляем до сотых и выводим
print(f"Доля совпадений: {match_ratio:.2f}")

Доля совпадений: 0.20


In [10]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Вычисляем RMSE и MAE
rmse = mean_squared_error(cold_user_recs["rating"], cold_user_recs["avg_rating"], squared=False)
mae = mean_absolute_error(cold_user_recs["rating"], cold_user_recs["avg_rating"])

# Выводим округленные результаты
print(round(rmse, 2), round(mae, 2))

0.78 0.62


In [11]:
# посчитаем покрытие холодных пользователей рекомендациями

cold_users_hit_ratio = cold_users_events_with_recs.groupby("user_id").agg(hits=("avg_rating", lambda x: (~x.isnull()).mean()))

print(f"Доля пользователей без релевантных рекомендаций: {(cold_users_hit_ratio == 0).mean().iat[0]:.2f}")
print(f"Среднее покрытие пользователей: {cold_users_hit_ratio[cold_users_hit_ratio != 0].mean().iat[0]:.2f}")

Доля пользователей без релевантных рекомендаций: 0.59
Среднее покрытие пользователей: 0.44


# === Базовые подходы: коллаборативная фильтрация

In [12]:
# Количество уникальных пользователей и книг
num_users = events["user_id"].nunique()
num_items = events["item_id"].nunique()

# Количество известных оценок (количество строк в events)
num_ratings = len(events)

# Вычисляем разреженность
sparsity = 1 - (num_ratings / (num_users * num_items))

# Округляем и выводим
print(f"Степень разреженности U-I матрицы: {round(sparsity, 4)}")

Степень разреженности U-I матрицы: 0.9993


In [13]:
from surprise import Dataset, Reader
from surprise import SVD

# используем Reader из библиотеки surprise для преобразования событий (events)
# в формат, необходимый surprise
reader = Reader(rating_scale=(1, 5))
surprise_train_set = Dataset.load_from_df(events_train[['user_id', 'item_id', 'rating']], reader)
surprise_train_set = surprise_train_set.build_full_trainset()

# инициализируем модель
svd_model = SVD(n_factors=100, random_state=0)

# обучаем модель
svd_model.fit(surprise_train_set)

In [14]:
surprise_test_set = list(events_test[['user_id', 'item_id', 'rating']].itertuples(index=False))

# получаем рекомендации для тестовой выборки
svd_predictions = svd_model.test(surprise_test_set)

In [15]:
from surprise import accuracy

rmse = accuracy.rmse(svd_predictions)
mae = accuracy.mae(svd_predictions)
                     
print(rmse, mae)

RMSE: 0.8259
MAE:  0.6458
0.8259326543219302 0.6458324731790988


In [16]:
from surprise import NormalPredictor

# инициализируем состояние генератора, это необходимо для получения
# одной и той же последовательности случайных чисел, только в учебных целях
np.random.seed(0)

random_model = NormalPredictor()

random_model.fit(surprise_train_set)
random_predictions = random_model.test(surprise_test_set)

In [17]:
from surprise import accuracy

# Вычисляем MAE для случайных предсказаний
mae_random = accuracy.mae(random_predictions)

mae_svd = accuracy.mae(svd_predictions)

# Вычисляем процентное превышение
mae_increase_percent = ((mae_random - mae_svd) / mae_svd) * 100

# Округляем до целых и выводим
print(f"MAE случайных рекомендаций выше на {round(mae_increase_percent)}%")

MAE:  0.9999
MAE:  0.6458
MAE случайных рекомендаций выше на 55%


In [18]:
from surprise import Dataset, Reader

def get_recommendations_svd(user_id, all_items, events, model, include_seen=True, n=5):
    """Возвращает n рекомендаций для user_id на основе модели SVD"""
    
    # Получаем список всех уникальных книг
    all_items = set(events['item_id'].unique())
    
    # Учитываем флаг, стоит ли включать уже прочитанные книги в рекомендации
    if include_seen:
        items_to_predict = list(all_items)
    else:
        # Получаем список книг, которые пользователь уже читал
        seen_items = set(events[events['user_id'] == user_id]['item_id'].unique())
        
        # Выбираем только книги, которые пользователь ещё не читал
        items_to_predict = list(all_items - seen_items)
    
    # Получаем предсказанные оценки для выбранных книг
    predictions = [model.predict(uid=user_id, iid=item_id) for item_id in items_to_predict]
    
    # Сортируем по убыванию предсказанной оценки и выбираем n лучших
    recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:n]
    
    return pd.DataFrame([(pred.iid, pred.est) for pred in recommendations], columns=["item_id", "score"])

In [19]:
user_id = 1296647  # пример ID пользователя
recommendations = get_recommendations_svd(user_id, events['item_id'].unique(), events, svd_model, include_seen=False, n=10)
print(recommendations)

    item_id     score
0     24812  5.000000
1    323355  4.988212
2  30688013  4.975064
3  11737700  4.972143
4   6898978  4.960511
5  22037424  4.955417
6     54741  4.937108
7   5031805  4.929516
8  18713259  4.924714
9  10954417  4.898045


In [20]:
get_recommendations_svd(1296647, items, events_test, svd_model)

,item_id,score
0,323355,4.988212
1,30688013,4.975064
2,11737700,4.972143
3,6898978,4.960511
4,22037424,4.955417


In [21]:
# выберем произвольного пользователя из тренировочной выборки ("прошлого")
user_id = events_train['user_id'].sample().iat[0]

print(f"user_id: {user_id}")

print("История (последние события, recent)")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("item_id")[["author", "title", "genre_and_votes"]], on="item_id")
)
user_history_to_print = user_history[["author", "title", "started_at", "read_at", "rating", "genre_and_votes"]].tail(10)
display(user_history_to_print)

print("Рекомендации")
user_recommendations = get_recommendations_svd(user_id, items, events_train, svd_model)
user_recommendations = user_recommendations.merge(items[["item_id", "author", "title", "genre_and_votes"]], on="item_id")
display(user_recommendations)

user_id: 1109853
История (последние события, recent)


,author,title,started_at,read_at,rating,genre_and_votes
0,Patrick Rothfuss,"The Wise Man's Fear (The Kingkiller Chronicle,...",2015-06-17,2015-08-31,5,"{'Fantasy': 16491, 'Fiction': 2222, 'Fantasy-E..."
1,Brandon Sanderson,"The Way of Kings (The Stormlight Archive, #1)",2015-08-31,2015-10-30,5,"{'Fantasy': 14291, 'Fiction': 1623, 'Fantasy-E..."
2,Brandon Sanderson,"Words of Radiance (The Stormlight Archive, #2)",2015-10-30,2016-03-17,5,"{'Fantasy': 8542, 'Fiction': 872, 'Fantasy-Epi..."
3,Andy Weir,The Martian,2014-12-07,2014-12-11,4,"{'Science Fiction': 11966, 'Fiction': 8430}"
4,Ken Follett,"Fall of Giants (The Century Trilogy, #1)",2016-05-12,2016-08-30,4,"{'Historical-Historical Fiction': 4665, 'Ficti..."


Рекомендации


,item_id,score,author,title,genre_and_votes
0,24813,5.000000,Bill Watterson,The Calvin and Hobbes Tenth Anniversary Book,"{'Sequential Art-Comics': 784, 'Humor': 377, '..."
1,11221285,5.000000,Brandon Sanderson,"The Way of Kings, Part 2 (The Stormlight Archi...","{'Fantasy': 641, 'Fiction': 46, 'Fantasy-Epic ..."
2,19219646,5.000000,Wolfgang Herrndorf,Arbeit und Struktur,"{'Nonfiction': 25, 'European Literature-German..."
3,7864312,5.000000,José Antonio Cotrina,La sombra de la luna (El ciclo de la luna roja...,"{'Fantasy': 20, 'Young Adult': 7}"
4,33353628,4.959447,Pénélope Bagieu,"Culottées #2 (Culottées, #2)","{'Sequential Art-Bande DessinÃ©e': 108, 'Femin..."


In [22]:
import scipy
import sklearn.preprocessing

# перекодируем идентификаторы пользователей: 
# из имеющихся в последовательность 0, 1, 2, ...
user_encoder = sklearn.preprocessing.LabelEncoder()
user_encoder.fit(events["user_id"])
events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])

# перекодируем идентификаторы объектов: 
# из имеющихся в последовательность 0, 1, 2, ...
item_encoder = sklearn.preprocessing.LabelEncoder()
item_encoder.fit(items["item_id"])
items["item_id_enc"] = item_encoder.transform(items["item_id"])
events_train["item_id_enc"] = item_encoder.transform(events_train["item_id"])
events_test["item_id_enc"] = item_encoder.transform(events_test["item_id"])

/tmp/ipykernel_4295/1044897688.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
/tmp/ipykernel_4295/1044897688.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])
/tmp/ipykernel_4295/1044897688.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [23]:
max_item_id_enc = events_train["item_id_enc"].max()
print(max_item_id_enc)

43304


In [24]:
# Число уникальных пользователей в train
num_users_train = events_train["user_id_enc"].nunique()

# Число уникальных книг в train
num_items_train = events_train["item_id_enc"].nunique()

# Общий размер в байтах
total_size_bytes = num_users_train * num_items_train

# Переводим в гигабайты (1 ГБ = 10^9 байт)
total_size_gb = total_size_bytes // 10**9  # Отбрасываем дробную часть

print(int(total_size_gb))

17


In [25]:
# создаём sparse-матрицу формата CSR 
user_item_matrix_train = scipy.sparse.csr_matrix((
    events_train["rating"],
    (events_train['user_id_enc'], events_train['item_id_enc'])),
    dtype=np.int8)

In [26]:
from implicit.als import AlternatingLeastSquares

als_model = AlternatingLeastSquares(factors=50, iterations=50, regularization=0.05, random_state=0)
als_model.fit(user_item_matrix_train)

/home/mle-user/mle_projects/mle-recsys-start/.env_recsys_start/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mle-user/mle_projects/mle-recsys-start/.env_recsys_start/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████| 50/50 [03:12<00:00,  3.86s/it]


In [27]:
def get_recommendations_als(user_item_matrix, model, user_id, user_encoder, item_encoder, include_seen=True, n=5):
    """
    Возвращает отранжированные рекомендации для заданного пользователя
    """
    user_id_enc = user_encoder.transform([user_id])[0]
    recommendations = model.recommend(
         user_id_enc, 
         user_item_matrix[user_id_enc], 
         filter_already_liked_items=not include_seen,
         N=n)
    recommendations = pd.DataFrame({"item_id_enc": recommendations[0], "score": recommendations[1]})
    recommendations["item_id"] = item_encoder.inverse_transform(recommendations["item_id_enc"])
    
    return recommendations

In [28]:
import random

# Выбираем случайного пользователя
random_user_id = random.choice(events_train["user_id"].unique())

# Получаем историю взаимодействий пользователя (с именами авторов и книг)
user_history = events_train[events_train["user_id"] == random_user_id][["item_id"]].merge(
    items[["item_id", "author", "title"]], on="item_id", how="left"
)

# Получаем рекомендации для пользователя
recommendations = get_recommendations_als(
    user_item_matrix=user_item_matrix_train,
    model=als_model,
    user_id=random_user_id,
    user_encoder=user_encoder,
    item_encoder=item_encoder,
    include_seen=False,
    n=10  # Количество рекомендаций
)

# Добавляем информацию об авторах и названиях книг
recommendations = recommendations.merge(
    items[["item_id", "author", "title"]], on="item_id", how="left"
)

# Добавляем признак "seen" (взаимодействовал ли уже пользователь с книгой)
recommendations["seen"] = recommendations["item_id"].isin(user_history["item_id"])

# Выводим результаты
#import ace_tools as tools

#tools.display_dataframe_to_user(name="История и рекомендации пользователя", dataframe=recommendations)

In [29]:
# получаем список всех возможных user_id (перекодированных)
user_ids_encoded = range(len(user_encoder.classes_))

# получаем рекомендации для всех пользователей
als_recommendations = als_model.recommend(
    user_ids_encoded, 
    user_item_matrix_train[user_ids_encoded], 
    filter_already_liked_items=False, N=100)

In [30]:
# преобразуем полученные рекомендации в табличный формат
item_ids_enc = als_recommendations[0]
als_scores = als_recommendations[1]

als_recommendations = pd.DataFrame({
    "user_id_enc": user_ids_encoded,
    "item_id_enc": item_ids_enc.tolist(), 
    "score": als_scores.tolist()})
als_recommendations = als_recommendations.explode(["item_id_enc", "score"], ignore_index=True)

# приводим типы данных
als_recommendations["item_id_enc"] = als_recommendations["item_id_enc"].astype("int")
als_recommendations["score"] = als_recommendations["score"].astype("float")

# получаем изначальные идентификаторы
als_recommendations["user_id"] = user_encoder.inverse_transform(als_recommendations["user_id_enc"])
als_recommendations["item_id"] = item_encoder.inverse_transform(als_recommendations["item_id_enc"])
als_recommendations = als_recommendations.drop(columns=["user_id_enc", "item_id_enc"])

In [31]:
als_recommendations = als_recommendations[["user_id", "item_id", "score"]]
als_recommendations.to_parquet("als_recommendations.parquet")

In [32]:
als_recommendations = (
    als_recommendations
    .merge(events_test[["user_id", "item_id", "rating"]]
               .rename(columns={"rating": "rating_test"}), 
           on=["user_id", "item_id"], how="left")
)

In [33]:
import sklearn.metrics

def compute_ndcg(rating: pd.Series, score: pd.Series, k):

    """ подсчёт ndcg
    rating: истинные оценки
    score: оценки модели
    k: количество айтемов (по убыванию score) для оценки, остальные - отбрасываются
    """
    
    # если кол-во объектов меньше 2, то NDCG - не определена
    if len(rating) < 2:
        return np.nan

    ndcg = sklearn.metrics.ndcg_score(np.asarray([rating.to_numpy()]), np.asarray([score.to_numpy()]), k=k)

    return ndcg

In [34]:
rating_test_idx = ~als_recommendations["rating_test"].isnull()
ndcg_at_5_scores = als_recommendations[rating_test_idx].groupby("user_id").apply(lambda x: compute_ndcg(x["rating_test"], x["score"], k=5))

In [35]:
print(ndcg_at_5_scores.mean())

0.975946709792109


# === Базовые подходы: контентные рекомендации

In [36]:
items["genre_and_votes"] = items["genre_and_votes"].apply(eval)

In [37]:
def get_genres(items):

    """ 
    извлекает список жанров по всем книгам, 
    подсчитывает долю голосов по каждому их них
    """
    
    genres_counter = {}
    
    for k, v, in items.iterrows():
        genre_and_votes = v.get("genre_and_votes", None)
        if genre_and_votes is None or not isinstance(genre_and_votes, dict):
            continue
        for genre, votes in genre_and_votes.items():
            # увеличиваем счётчик жанров
            try:
                genres_counter[genre] += votes
            except KeyError:
                genres_counter[genre] = 0

    genres = pd.Series(genres_counter, name="votes")
    genres = genres.to_frame()
    genres = genres.reset_index().rename(columns={"index": "name"})
    genres.index.name = "genre_id"
    
    return genres
   
genres = get_genres(items)

In [38]:
genres["score"] = genres["votes"] / genres["votes"].sum()
genres.sort_values(by="score", ascending=False).head(10)

,name,votes,score
genre_id,,,
25,Fantasy,6850060,0.149651
1,Fiction,6406256,0.139955
38,Classics,3414934,0.074605
18,Young Adult,3296951,0.072027
34,Romance,2422614,0.052926
5,Nonfiction,1737406,0.037957
16,Historical-Historical Fiction,1531205,0.033452
20,Mystery,1371196,0.029956
24,Science Fiction,1218917,0.026629


In [39]:
def get_item2genre_matrix(genres, items):

    genre_names_to_id = genres.reset_index().set_index("name")["genre_id"].to_dict()
    
    # list to build CSR matrix
    genres_csr_data = []
    genres_csr_row_idx = []
    genres_csr_col_idx = []
    
    for item_idx, (k, v) in enumerate(items.iterrows()):
        if v["genre_and_votes"] is None:
            continue
        for genre_name, votes in v["genre_and_votes"].items():
            genre_idx = genre_names_to_id[genre_name]
            genres_csr_data.append(int(votes))
            genres_csr_row_idx.append(item_idx)
            genres_csr_col_idx.append(genre_idx)

    genres_csr = scipy.sparse.csr_matrix((genres_csr_data, (genres_csr_row_idx, genres_csr_col_idx)), shape=(len(items), len(genres)))
    # нормализуем, чтобы сумма оценок принадлежности к жанру была равна 1
    genres_csr = sklearn.preprocessing.normalize(genres_csr, norm='l1', axis=1)
    
    return genres_csr

In [40]:
items = items.sort_values(by="item_id_enc")
all_items_genres_csr = get_item2genre_matrix(genres, items)

In [41]:
user_id = 1000010
user_events = events_train.query("user_id == @user_id")[["item_id", "rating"]]
user_items = items[items["item_id"].isin(user_events["item_id"])]

user_items_genres_csr = get_item2genre_matrix(genres, user_items)
user_items_genres_csr

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 149 stored elements and shape (22, 815)>

In [42]:
# вычислим склонность пользователя к жанрам как среднее взвешенное значение популяции на его оценки книг.

# преобразуем пользовательские оценки из списка в вектор-столбец
user_ratings = user_events["rating"].to_numpy() / 5
user_ratings = np.expand_dims(user_ratings, axis=1)

user_items_genres_weighted = user_items_genres_csr.multiply(user_ratings)

user_genres_scores = np.asarray(user_items_genres_weighted.mean(axis=0))

In [43]:
# выведем список жанров, которые предпочитает пользователь

user_genres = genres.copy()
user_genres["score"] = np.ravel(user_genres_scores)
user_genres = user_genres[user_genres["score"] > 0].sort_values(by=["score"], ascending=False)

user_genres.head(5)

,name,votes,score
genre_id,,,
1,Fiction,6406256,0.180513
38,Classics,3414934,0.098675
25,Fantasy,6850060,0.088185
5,Nonfiction,1737406,0.044133
24,Science Fiction,1218917,0.041981


In [44]:
from sklearn.metrics.pairwise import cosine_similarity

# вычисляем сходство между вектором пользователя и векторами по книгам
similarity_scores = cosine_similarity(all_items_genres_csr, user_genres_scores)

# преобразуем в одномерный массив
similarity_scores = similarity_scores.flatten()

# получаем индексы top-k (по убыванию значений), по сути, индексы книг (encoded)
k = 5
top_k_indices = np.argsort(similarity_scores)[-k:][::-1]

In [45]:
selected_items = items[items["item_id_enc"].isin(top_k_indices)]

with pd.option_context("max_colwidth", 100):
   display(selected_items[["author", "title", "genre_and_votes"]])

,author,title,genre_and_votes
564712,Ray Bradbury,"Farewell Summer (Green Town, #3)","{'Fiction': 170, 'Fantasy': 72, 'Science Fiction': 72, 'Classics': 52}"
80465,G.K. Chesterton,The Napoleon of Notting Hill,"{'Fiction': 166, 'Classics': 88, 'Fantasy': 44, 'Humor': 22, 'Literature': 20}"
1168335,Ray Bradbury,"Dandelion Wine (Green Town, #1)","{'Fiction': 1438, 'Classics': 914, 'Science Fiction': 529, 'Fantasy': 456, 'Young Adult': 212}"
393210,"G.K. Chesterton, Jonathan Lethem",The Man Who Was Thursday: A Nightmare,"{'Fiction': 1257, 'Classics': 929, 'Mystery': 469, 'Fantasy': 293, 'Philosophy': 156, 'Literatur..."
2244467,Samuel Butler,"Erewhon (Erewhon , #1)","{'Fiction': 162, 'Classics': 139, 'Science Fiction': 60, 'Fantasy': 55}"


# === Базовые подходы: валидация

In [46]:
als_recommendations=pd.read_parquet("als_recommendations.parquet")

In [47]:
def process_events_recs_for_binary_metrics(events_train, events_test, recs, top_k=None):

    """
    размечает пары <user_id, item_id> для общего множества пользователей признаками
    - gt (ground truth)
    - pr (prediction)
    top_k: расчёт ведётся только для top k-рекомендаций
    """

    events_test["gt"] = True
    common_users = set(events_test["user_id"]) & set(recs["user_id"])

    print(f"Common users: {len(common_users)}")
    
    events_for_common_users = events_test[events_test["user_id"].isin(common_users)].copy()
    recs_for_common_users = recs[recs["user_id"].isin(common_users)].copy()

    recs_for_common_users = recs_for_common_users.sort_values(["user_id", "score"], ascending=[True, False])

    # оставляет только те item_id, которые были в events_train, 
    # т. к. модель не имела никакой возможности давать рекомендации для новых айтемов
    events_for_common_users = events_for_common_users[events_for_common_users["item_id"].isin(events_train["item_id"].unique())]

    if top_k is not None:
        recs_for_common_users = recs_for_common_users.groupby("user_id").head(top_k)
    
    events_recs_common = events_for_common_users[["user_id", "item_id", "gt"]].merge(
        recs_for_common_users[["user_id", "item_id", "score"]], 
        on=["user_id", "item_id"], how="outer")    

    events_recs_common["gt"] = events_recs_common["gt"].fillna(False)
    events_recs_common["pr"] = ~events_recs_common["score"].isnull()
    
    events_recs_common["tp"] = events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fp"] = ~events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fn"] = events_recs_common["gt"] & ~events_recs_common["pr"]

    return events_recs_common

In [48]:
events_recs_for_binary_metrics = process_events_recs_for_binary_metrics(
  events_train,
    events_test, 
    als_recommendations, 
    top_k=5)

/tmp/ipykernel_4295/3166051426.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["gt"] = True


Common users: 123223


In [49]:
def compute_cls_metrics(events_recs_for_binary_metric):
    
    groupper = events_recs_for_binary_metric.groupby("user_id")

    # precision = tp / (tp + fp)
    precision = groupper["tp"].sum()/(groupper["tp"].sum()+groupper["fp"].sum())
    precision = precision.fillna(0).mean()
    
    # recall = tp / (tp + fn)
    recall = groupper["tp"].sum() / (groupper["tp"].sum() + groupper["fn"].sum())  # ваш код здесь #
    recall = recall.fillna(0).mean()

    return precision, recall

In [50]:
try:
    # Вычисляем Precision@5 и Recall@5
    precision_at_5, recall_at_5 = compute_cls_metrics(events_recs_for_binary_metrics)

    # Выводим значение recall@5
    recall_at_5
except NameError as e:
    recall_at_5 = str(e)

recall_at_5

0.014121568795222568

# === Двухстадийный подход: метрики

In [51]:
# Количество уникальных объектов в рекомендациях
unique_recommended_items = als_recommendations["item_id"].nunique()

# Общее количество объектов в train
total_items_in_train = events_train["item_id"].nunique()

# Вычисляем покрытие
cov_items = unique_recommended_items / total_items_in_train

# Выводим результат с округлением до двух знаков
print(f"{cov_items:.2f}")

0.10


In [52]:
# Размечаем рекомендации признаком "прочитано" (read), используя events_train
events_train.loc[:, "read"] = True  # Указываем, что книга была прочитана

# Объединяем рекомендации с events_train, избегая дубликатов с помощью suffixes
als_recommendations = als_recommendations.merge(
    events_train[["user_id", "item_id", "read"]],
    on=["user_id", "item_id"],
    how="left",
    suffixes=(None, "_train")  # Предотвращаем создание read_x, read_y
)

# Заполняем пропуски False и приводим к булевому типу
als_recommendations["read"] = als_recommendations["read"].fillna(False).astype(bool)

# Сортируем рекомендации по пользователю и убыванию score
als_recommendations = als_recommendations.sort_values(["user_id", "score"], ascending=[True, False])
als_recommendations["rank"] = als_recommendations.groupby("user_id").cumcount() + 1

# Вычисляем Novelty@5 для каждого пользователя
novelty_5 = (1 - als_recommendations.query("rank <= 5").groupby("user_id")["read"].mean())

# Вычисляем среднее значение Novelty@5
average_novelty_5 = novelty_5.mean()

# Выводим результат
average_novelty_5


/tmp/ipykernel_4295/142976345.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_train.loc[:, "read"] = True  # Указываем, что книга была прочитана


0.607333279143491

In [53]:
als_recommendations.head()

,user_id,item_id,score,read,rank
0,1000000,3,0.990941,True,1
1,1000000,15881,0.896617,True,2
2,1000000,5,0.864404,True,3
3,1000000,6,0.822254,False,4
4,1000000,2,0.774095,True,5


# === Двухстадийный подход: модель

In [54]:
# Задаём точку разбиения
split_date_for_labels = pd.to_datetime("2017-09-15").date()

# Определяем индексы для разделения данных
split_date_for_labels_idx = events_test["started_at"] < split_date_for_labels

# Первая часть данных (таргеты) — события до split_date_for_labels
events_labels = events_test[split_date_for_labels_idx].copy()

# Вторая часть данных (новая тестовая выборка) — события после split_date_for_labels
events_test_2 = events_test[~split_date_for_labels_idx].copy()

In [55]:
num_unique_users_labels = events_labels["user_id"].nunique()
num_unique_users_labels

99849

In [56]:
# Загружаем рекомендации от двух базовых генераторов
als_recommendations = pd.read_parquet("als_recommendations.parquet")
content_recommendations = pd.read_parquet("candidates/training/content_recommendations.parquet")

# Удаляем дубликаты (если есть)
als_recommendations = als_recommendations.drop_duplicates(subset=["user_id", "item_id"])
content_recommendations = content_recommendations.drop_duplicates(subset=["user_id", "item_id"])

# Проверяем количество записей после удаления дубликатов
num_als = len(als_recommendations)
num_content = len(content_recommendations)

# Объединяем кандидатов по user_id и item_id
candidates = pd.merge(
    als_recommendations.rename(columns={"score": "als_score"}),
    content_recommendations.rename(columns={"score": "cnt_score"}),
    on=["user_id", "item_id"],
    how="outer"
)

# Проверяем количество записей в объединенной таблице
num_candidates = len(candidates)

# Выводим результаты
num_als, num_content, num_candidates

(43058500, 42822000, 83001774)

In [57]:
# Добавляем таргет к кандидатам:
# — 1 для тех item_id, которые пользователь прочитал (из events_labels)
# — 0 для всех остальных (по умолчанию)
events_labels["target"] = 1
candidates = candidates.merge(
    events_labels[["user_id", "item_id", "target"]], 
    on=["user_id", "item_id"],  # ваш код здесь #
    how="left"
)
candidates["target"] = candidates["target"].fillna(0).astype("int")

# В кандидатах оставляем только тех пользователей, у которых есть хотя бы один положительный таргет
candidates_to_sample = candidates.groupby("user_id").filter(lambda x: x["target"].sum() > 0)

# Для каждого пользователя оставляем только 4 негативных примера
negatives_per_user = 4
candidates_for_train = pd.concat([
    candidates_to_sample.query("target == 1"),  # ваш код здесь # (все положительные примеры)
    candidates_to_sample.query("target == 0")
        .groupby("user_id")
        .apply(lambda x: x.sample(min(len(x), negatives_per_user), random_state=0))  # Не менее 4 негативных примеров
]).reset_index(drop=True)

len(candidates_for_train)

213269

In [58]:
from catboost import CatBoostClassifier, Pool

# задаём имена колонок признаков и таргета
features = ['als_score', 'cnt_score']
target = 'target'

# Create the Pool object
train_data = Pool(
    data=candidates_for_train[features], 
    label=candidates_for_train[target])

# инициализируем модель CatBoostClassifier
cb_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    verbose=100,
    random_seed=0
)

# тренируем модель
cb_model.fit(train_data)

0:	learn: 0.6527339	total: 74.4ms	remaining: 1m 14s
100:	learn: 0.5120989	total: 1.96s	remaining: 17.5s
200:	learn: 0.5113937	total: 3.88s	remaining: 15.4s
300:	learn: 0.5107914	total: 5.85s	remaining: 13.6s
400:	learn: 0.5102344	total: 7.85s	remaining: 11.7s
500:	learn: 0.5098070	total: 9.84s	remaining: 9.8s
600:	learn: 0.5093402	total: 11.8s	remaining: 7.86s
700:	learn: 0.5089716	total: 13.8s	remaining: 5.89s
800:	learn: 0.5086073	total: 15.8s	remaining: 3.92s
900:	learn: 0.5083284	total: 17.7s	remaining: 1.95s
999:	learn: 0.5080276	total: 19.7s	remaining: 0us


In [59]:
# Загружаем рекомендации от двух базовых генераторов
als_recommendations_2 = pd.read_parquet("candidates/inference/als_recommendations.parquet")
content_recommendations_2 = pd.read_parquet("candidates/inference/content_recommendations.parquet")

# Объединяем рекомендации по user_id и item_id
candidates_to_rank = pd.merge(
    als_recommendations_2.rename(columns={"score": "als_score"}),
    content_recommendations_2.rename(columns={"score": "cnt_score"}),
    on=["user_id", "item_id"],  # ваш код здесь #
    how="outer"
)

# Оставляем только тех пользователей, что есть в тестовой выборке (для экономии ресурсов)
candidates_to_rank = candidates_to_rank[candidates_to_rank["user_id"].isin(events_test_2["user_id"].drop_duplicates())]

# Выводим количество записей
print(len(candidates_to_rank))

14517152


In [60]:
from catboost import Pool

# Создаем объект Pool для CatBoost
inference_data = Pool(data=candidates_to_rank[features])

# Получаем вероятности предсказаний модели
predictions = cb_model.predict_proba(inference_data)

# Записываем в candidates_to_rank только вероятность положительного класса
candidates_to_rank["cb_score"] = predictions[:, 1]

# Сортируем по user_id и cb_score (по убыванию)
candidates_to_rank = candidates_to_rank.sort_values(["user_id", "cb_score"], ascending=[True, False])

# Проставляем ранги, начиная с 1 (чем выше cb_score, тем ниже ранг)
candidates_to_rank["rank"] = candidates_to_rank.groupby("user_id").cumcount() + 1  # ваш код здесь #

# Ограничиваем количество рекомендаций для каждого пользователя
max_recommendations_per_user = 100
final_recommendations = candidates_to_rank[candidates_to_rank["rank"] <= max_recommendations_per_user]  # ваш код здесь #

# Выводим финальные рекомендации
len(final_recommendations)

7519400

In [61]:
# Объединяем тренировочные события и таргеты для финальной оценки
events_inference = pd.concat([events_train, events_labels])

# Вызываем функцию для обработки рекомендаций
cb_events_recs_for_binary_metrics_5 = process_events_recs_for_binary_metrics(
    events_inference,
    events_test_2,  # ваш код здесь #
    final_recommendations.rename(columns={"cb_score": "score"}), 
    top_k=5
)

# Вычисляем Precision@5 и Recall@5
cb_precision_5, cb_recall_5 = compute_cls_metrics(cb_events_recs_for_binary_metrics_5)  # ваш код здесь #

# Выводим метрики
print(f"precision: {cb_precision_5:.3f}, recall: {cb_recall_5:.3f}")

Common users: 75194
precision: 0.006, recall: 0.015


# === Двухстадийный подход: построение признаков

In [62]:
import numpy as np

# Вычисляем "возраст" книги (age) на основе года публикации
items["age"] = 2018 - items["publication_year"]

# Обрабатываем некорректные значения (например, если год публикации отрицательный)
invalid_age_idx = items["age"] < 0
items.loc[invalid_age_idx, "age"] = np.nan

# Приводим возраст к float
items["age"] = items["age"].astype("float")

# Добавляем признаки "age" и "average_rating" к кандидатам для тренировки модели
candidates_for_train = candidates_for_train.merge(
    items[["item_id", "age", "average_rating"]],  # ваш код здесь #
    on="item_id",
    how="left"
)

# Добавляем признаки "age" и "average_rating" к кандидатам для ранжирования
candidates_to_rank = candidates_to_rank.merge(
    items[["item_id", "age", "average_rating"]],  # ваш код здесь #
    on="item_id",
    how="left"
)
# Вычисляем медианный возраст книги для candidates_to_rank
median_age_candidates_to_rank = candidates_to_rank["age"].median()
median_age_candidates_to_rank

7.0

In [63]:
# Определяем функцию для вычисления пользовательских признаков
def get_user_features(events):
    """ Считает пользовательские признаки """
    
    user_features = events.groupby("user_id").agg(
        reading_years=("started_at", lambda x: (x.max() - x.min()).days / 365.25),
        books_read=("user_id", "count"),  # ваш код здесь #
        rating_avg=("rating", "mean"),
        rating_std=("rating", "std")
    )
    
    user_features["books_per_year"] = user_features["books_read"] / user_features["reading_years"]
    
    return user_features

# Вычисляем пользовательские признаки для обучения
user_features_for_train = get_user_features(events_train)
candidates_for_train = candidates_for_train.merge(user_features_for_train, on="user_id", how="left")

# Оставляем только тех пользователей, которые есть в тесте
events_inference = pd.concat([events_train, events_labels])
events_inference = events_inference[events_inference["user_id"].isin(events_test["user_id"].drop_duplicates())]

# Вычисляем пользовательские признаки для ранжирования
user_features_for_ranking = get_user_features(events_inference)  # ваш код здесь #
candidates_to_rank = candidates_to_rank.merge(user_features_for_ranking, on="user_id", how="left")  # ваш код здесь #

# Вычисляем медиану количества прочитанных книг в candidates_for_train
median_books_read = candidates_for_train["books_read"].median()
median_books_read

32.0

In [64]:
# Определяем топ-10 популярных жанров по названию, а не по индексу
genres_top_k = 10
top_genres = genres.sort_values("votes", ascending=False).head(genres_top_k)
genres_top_idx = top_genres.index  # Индексы топовых жанров
genres_top_names = top_genres["name"].tolist()  # Названия топовых жанров

# Определяем "остальные жанры"
genres_others_idx = list(set(genres.index) - set(genres_top_idx))
genres_others_column = "genre_others"

# Формируем список названий жанровых столбцов
genres_top_columns = [f"genre_{name}" for name in genres_top_names]
genre_columns = genres_top_columns + [genres_others_column]

# Создаём таблицу принадлежности книг к жанрам с правильными названиями
item_genres = pd.concat([
        # Для топ-10 жанров используем реальные названия
        pd.DataFrame(all_items_genres_csr[:, genres_top_idx].toarray(), columns=genres_top_columns),
        # Для остальных жанров суммируем значения
        pd.DataFrame(all_items_genres_csr[:, genres_others_idx].sum(axis=1), columns=[genres_others_column])
    ],
    axis=1
).reset_index().rename(columns={"index": "item_id_enc"})

# Объединяем с books, избегая дублирования столбцов
items = items.merge(item_genres, on="item_id_enc", how="left", suffixes=("", "_duplicate"))

# Удаляем ненужные дублирующиеся столбцы
duplicate_columns = [col for col in items.columns if col.endswith("_duplicate")]
items.drop(columns=duplicate_columns, inplace=True)

# Проверяем, что теперь жанры есть в items
print("Столбцы после исправления:", items.columns)

# Проверяем, что ожидаемые жанры теперь присутствуют
missing_genres = [col for col in genre_columns if col not in items.columns]
if missing_genres:
    print("Отсутствующие жанры после исправления:", missing_genres)
else:
    print("Все жанры добавлены корректно!")

# Определяем функцию вычисления средних жанровых предпочтений пользователя
def get_user_genres(events, items, item_genre_columns):
    user_genres = (
        events
        .merge(items[["item_id"] + item_genre_columns], on="item_id", how="left")
        .groupby("user_id")[item_genre_columns].mean()
    )
    return user_genres

# Вычисляем пользовательские жанровые предпочтения для обучения
user_genres_for_train = get_user_genres(events_train, items, genre_columns)  # ваш код здесь #
candidates_for_train = candidates_for_train.merge(user_genres_for_train, on="user_id", how="left")

# Вычисляем пользовательские жанровые предпочтения для ранжирования
user_genres_for_ranking = get_user_genres(events_inference, items, genre_columns)
candidates_to_rank = candidates_to_rank.merge(user_genres_for_ranking, on="user_id", how="left")

# Вычисляем медиану жанровости книг в candidates_for_train для жанра "Romance"
median_romance_genre = candidates_for_train["genre_Romance"].median()
round(median_romance_genre, 2)

Столбцы после исправления: Index(['item_id', 'author', 'title', 'description', 'genre_and_votes',
       'num_pages', 'average_rating', 'ratings_count', 'text_reviews_count',
       'publisher', 'publication_year', 'country_code', 'language_code',
       'format', 'is_ebook', 'isbn', 'isbn13', 'genre_and_votes_dict',
       'genre_and_votes_str', 'item_id_enc', 'age', 'genre_Fantasy',
       'genre_Fiction', 'genre_Classics', 'genre_Young Adult', 'genre_Romance',
       'genre_Nonfiction', 'genre_Historical-Historical Fiction',
       'genre_Mystery', 'genre_Science Fiction', 'genre_Fantasy-Paranormal',
       'genre_others'],
      dtype='object')
Все жанры добавлены корректно!


0.04

In [65]:
from catboost import CatBoostClassifier, Pool

# задаём имена колонок признаков и таргета
features = ['als_score', 'cnt_score', 
    'age', 'average_rating', 'reading_years', 'books_read', 
    'rating_avg', 'rating_std', 
    'books_per_year'] + genre_columns
target = 'target'

# создаём Pool
train_data = Pool(
    data=candidates_for_train[features], 
    label=candidates_for_train[target])

# инициализируем модель CatBoostClassifier
cb_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    verbose=100,
    random_seed=0,
)

# тренируем модель
cb_model.fit(train_data)

0:	learn: 0.6468075	total: 31.3ms	remaining: 31.2s
100:	learn: 0.4657324	total: 2.72s	remaining: 24.2s
200:	learn: 0.4574211	total: 5.36s	remaining: 21.3s
300:	learn: 0.4514006	total: 8.01s	remaining: 18.6s
400:	learn: 0.4465119	total: 10.6s	remaining: 15.9s
500:	learn: 0.4422618	total: 13.3s	remaining: 13.3s
600:	learn: 0.4385492	total: 16.1s	remaining: 10.7s
700:	learn: 0.4351416	total: 18.7s	remaining: 7.99s
800:	learn: 0.4318738	total: 21.4s	remaining: 5.33s
900:	learn: 0.4286755	total: 24.2s	remaining: 2.65s
999:	learn: 0.4256539	total: 26.8s	remaining: 0us


In [66]:
from catboost import Pool

# Создаем объект Pool для CatBoost
inference_data = Pool(data=candidates_to_rank[features])

# Получаем вероятности предсказаний модели
predictions = cb_model.predict_proba(inference_data)

# Используем только вероятность положительного класса (второй столбец)
candidates_to_rank["cb_score"] = predictions[:, 1]  # ваш код здесь #

# Сортируем по user_id и cb_score (по убыванию)
candidates_to_rank = candidates_to_rank.sort_values(["user_id", "cb_score"], ascending=[True, False])

# Проставляем ранги, начиная с 1 (чем выше cb_score, тем ниже ранг)
candidates_to_rank["rank"] = candidates_to_rank.groupby("user_id").cumcount() + 1  # ваш код здесь #

# Ограничиваем количество рекомендаций до 100 на пользователя
max_recommendations_per_user = 100
final_recommendations = candidates_to_rank.query("rank <= @max_recommendations_per_user")

In [67]:
num_users_final_recommendations = final_recommendations["user_id"].nunique()
print(num_users_final_recommendations)

75194


In [68]:
final_recommendations.to_parquet("final_recommendations_feat.parquet", index=False)
print("Файл final_recommendations_feat.parquet успешно сохранён.")

Файл final_recommendations_feat.parquet успешно сохранён.


In [69]:
# Объединяем тренировочные события и таргеты для финальной оценки
events_inference = pd.concat([events_train, events_labels])

# Оставляем только пользователей из тестовой выборки
events_inference = events_inference[events_inference["user_id"].isin(events_test_2["user_id"].drop_duplicates())]

# Вычисляем precision и recall для CatBoost рекомендаций
cb_events_recs_for_binary_metrics_5 = process_events_recs_for_binary_metrics(
    events_inference,
    events_test_2,  
    final_recommendations.rename(columns={"cb_score": "score"}), 
    top_k=5  
)

# Вычисляем Precision@5 и Recall@5
cb_precision_5, cb_recall_5 = compute_cls_metrics(cb_events_recs_for_binary_metrics_5)

# Выводим результаты
print(f"precision: {cb_precision_5:.3f}, recall: {cb_recall_5:.3f}")

Common users: 75194
precision: 0.012, recall: 0.030


In [70]:
# Получаем важность признаков из модели CatBoost
feature_importance = pd.DataFrame(cb_model.get_feature_importance(), 
                                  index=features, 
                                  columns=["fi"])

# Сортируем признаки по убыванию важности
feature_importance = feature_importance.sort_values(by="fi", ascending=False)  # ваш код здесь #

# Выводим таблицу важности признаков
print(feature_importance)

                                            fi
als_score                            25.936991
age                                  19.373863
average_rating                       14.777075
books_read                            7.064894
cnt_score                             3.746944
reading_years                         3.109407
genre_Fiction                         2.748945
genre_others                          2.689428
genre_Young Adult                     2.613080
genre_Fantasy                         2.377939
genre_Romance                         2.330318
books_per_year                        1.878157
genre_Mystery                         1.666996
genre_Fantasy-Paranormal              1.543947
rating_avg                            1.532771
genre_Classics                        1.528630
genre_Science Fiction                 1.434332
genre_Historical-Historical Fiction   1.409797
genre_Nonfiction                      1.151471
rating_std                            1.085015
